In [5]:
from sklearn.datasets import load_digits
from sklearn.decomposition import FastICA
from sklearn.decomposition import PCA
import pickle5 as pickle
import copy

import pandas as pd
import seaborn as sns
import numpy
import torch
import scipy
import scipy.stats
import numpy as np

%matplotlib widget
import matplotlib.pyplot as plt

In [34]:
import os
dim=3
base_dir='/Users/phumon/Documents/Research/nlu-test-sets/params_mvirt_sync'
file_name=f'lr-0.0001-steps-$4000-alpha-lognormal-identity-dim8_theta-normal-identity_nosubsample_1.00_0.40_particles8/'
exp_dir=os.path.join(base_dir, file_name)

print(exp_dir)


/Users/phumon/Documents/Research/nlu-test-sets/params_mvirt_sync/lr-0.0001-steps-$4000-alpha-lognormal-identity-dim8_theta-normal-identity_nosubsample_1.00_0.40_particles8/


In [35]:
param_plot_stats = {}
keys = ['a', 'b', 'g', 't']
for key in keys:
    with open(os.path.join(exp_dir, 'plot_stats_pickles', f'{key}.p'), 'rb') as f:
        param_plot_stats[key] = pickle.load(f)

In [62]:

param_plot_stats['a']['log_mean'] = param_plot_stats['a']['mean'].apply(lambda x: numpy.log(x))
alpha_mean, alpha_var = param_plot_stats['a']["log_mean"].to_numpy().mean(), param_plot_stats['a']["var"].to_numpy().mean()
print("alpha mean: ", alpha_mean)
print("alpha var: ", alpha_var)
print("beta mean: ", beta_mean)
print("beta var: ", beta_var)
print("theta mean: ", t_mean)
print("theta var: ", t_var)

alpha mean:  [0.0565874  0.05625768 0.05636056 0.05651398 0.05657453 0.05659167
 0.05667934 0.05645685]
alpha var:  [0.18627433 0.18614232 0.18624339 0.18630315 0.18632454 0.18627604
 0.18635483 0.18626695]
beta mean:  [-0.01541819 -0.01563922 -0.0156223  -0.01538601 -0.01517688 -0.01579989
 -0.01571705 -0.01565638]
beta var:  [0.91355278 0.91321918 0.9131526  0.91358529 0.91328188 0.91342821
 0.913385   0.913155  ]
theta mean:  [-0.01052167 -0.01101482 -0.0110263  -0.01047297 -0.01245633 -0.0095041
 -0.012507   -0.01192814]
theta var:  [0.98810289 0.98602625 0.98624028 0.98607352 0.98780786 0.98967992
 0.98757252 0.98782954]


In [54]:
beta_mean, beta_var = param_plot_stats['b']["mean"].to_numpy().mean(), param_plot_stats['b']["var"].to_numpy().mean()
print("posterior beta mean: ", beta_mean.mean())
print("posterior beta var: ", beta_var.mean())

posterior beta mean:  -0.015551991250365972
posterior beta var:  0.9133449924513697


In [51]:
t_mean, t_var = param_plot_stats['t']["mean"].to_numpy().mean(), param_plot_stats['t']["var"].to_numpy().mean()
print("posterior theta mean: ", t_mean.mean())
print("posterior theta var: ", t_var.mean())

posterior theta mean:  -0.011178915586949896
posterior theta var:  0.9874166001876196


In [37]:
dimensions=3
for dim in range(dimensions):
    param_plot_stats['a']['log_mean_'+str(dim)]= param_plot_stats['a']['log_mean'].apply(lambda x: x[dim])
    param_plot_stats['b']['mean_'+str(dim)]= param_plot_stats['b']['mean'].apply(lambda x: x[dim])

In [38]:
plt.figure()
for dim in range(dimensions):
    plt.plot(param_plot_stats['a']['log_mean_'+str(dim)], param_plot_stats['b']['mean_'+str(dim)], marker="o", linestyle="", label=dim)

plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
plt.figure()
plt.plot(param_plot_stats['a']['log_mean_0'], param_plot_stats['a']['log_mean_1'], marker="o", linestyle="", label=1)
plt.plot(param_plot_stats['a']['log_mean_0'], param_plot_stats['a']['log_mean_2'], marker="o", linestyle="", label=2)

plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
param_plot_stats['a']['log_mean'][0]

array([0.03206562, 0.02932311, 0.03850344, 0.03990555, 0.03952771,
       0.04200046, 0.03938458, 0.0351401 ])

https://stackoverflow.com/questions/35407669/independent-component-analysis-ica-in-python

In [41]:
alpha_arr = np.vstack(param_plot_stats['b']['mean'])
pca = PCA(n_components=2, random_state=0)
new_alpha_ = pca.fit_transform(alpha_arr)  # Reconstruct signals
#A_alpha_ = ica.mixing_  # Get estimated mixing matrix


In [42]:
print(new_alpha_.shape)

(2000, 2)


In [43]:
from multi_virt_v2 import *

ModuleNotFoundError: No module named 'multi_virt_v2'

In [ ]:
repo = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

In [ ]:
#datasets="boolq,cb,commonsenseqa,copa,cosmosqa,hellaswag,rte,snli,wic,qamr,arct,mcscript,mctaco,mutual,mutual-plus,quoref,socialiqa,squad-v2,wsc,mnli,mrqa-nq,newsqa,abductive-nli,arc-easy,arc-challenge,piqa,quail,winogrande,anli"
datasets="snli"
data_names, responses, n_items = get_files(
    os.path.join(repo, 'data'),
    "csv",
    set(datasets.split(','))
)

In [ ]:
task_metadata = pd.read_csv('../irt_scripts/task_metadata.csv')
task_metadata.set_index("jiant_name", inplace=True)
task_list = [x for x in task_metadata.index if x in data_names]

In [ ]:
total = 0
task_name = []
task_format = []

for tname, size in zip(data_names, n_items):
    name = task_metadata.loc[tname]['taskname']
    total += size
    task_name += [name for _ in range(size)]
    task_format += [task_metadata.loc[tname]['format'] for _ in range(size)]
    
task_name = pd.DataFrame(task_name, columns=['task_name'])
task_format = pd.DataFrame(task_format, columns=['format'])
task_name_format = pd.concat([task_name, task_format], axis=1)

In [ ]:
task_name_format

In [ ]:

param = pd.concat([pd.DataFrame(new_alpha_.tolist()), task_name_format], axis=1)

In [ ]:
param

In [ ]:


plt.figure()
plt.plot(param[0], param[1], "kx")
plt.show()




In [ ]:
groups = param.groupby("task_name")
groups

In [ ]:
plt.figure()

groups = param.groupby("format")
for name, group in groups:
    print("name: ",name)
    plt.plot(group[0], group[1], marker="o", linestyle="")
plt.legend()
plt.show()

In [ ]:
param[0]

# Theta

In [ ]:
beta_arr = np.vstack(param_plot_stats['b']['mean'])
pca = PCA(n_components=2, random_state=0)
new_beta_ = pca.fit_transform(beta_arr)  # Reconstruct signals
#A_alpha_ = ica.mixing_  # Get estimated mixing matrix


In [ ]:
params_beta = pd.concat([pd.DataFrame(new_beta_.tolist()), task_name_format], axis=1)

In [ ]:
groups = params_beta.groupby("task_name")


In [ ]:
plt.figure()
for name, group in groups:
    print("name: ",name)
    plt.plot(group[0], group[1], marker="o", linestyle="", label=name)
plt.legend()
plt.show()

In [ ]:
# END scatter plot

In [ ]:
beta_arr = np.vstack(param_plot_stats['b']['mean'])
ica = FastICA(n_components=3, random_state=0)
S_beta_ = ica.fit_transform(beta_arr)  # Reconstruct signals
A_beta_ = ica.mixing_  # Get estimated mixing matrix


In [ ]:
models = [beta_arr, S_beta_]
names = ['Alpha (mixed signal)',
         'ICA recovered Alpha signals']
colors = ['red', 'steelblue', 'orange']

plt.figure()
for ii, (model, name) in enumerate(zip(models, names), 1):
    plt.subplot(2, 1, ii)
    plt.title(name)
    for sig, color in zip(model.T, colors):
        plt.plot(sig, color=color)

plt.subplots_adjust(0.09, 0.04, 0.94, 0.94, 0.26, 0.46)
plt.show()

In [ ]:
params_beta = pd.concat([pd.DataFrame(S_beta_.tolist()), task_name_format], axis=1)

In [ ]:
params_beta

In [ ]:
groups = params_beta.groupby("task_name")
groups

In [ ]:
plt.figure()
for name, group in groups:
    print("name: ",name)
    plt.plot(group[0], group[1], marker="o", linestyle="", label=1)
    plt.plot(group[0], group[2], marker="o", linestyle="", label=2)
plt.legend()
plt.show()